In [ ]:
%load_ext autoreload
%autoreload 2

import dotenv
import langchain
from interlab import actor, tracing
from interlab.tracing import TracingNode
from interlab.environment import BaseEnvironment, DelayedEnvironment
from interlab.actor.affordance import Affordance, query_for_action
from interlab.actor import OneShotLLMActor
from pydantic.dataclasses import dataclass, Field

dotenv.load_dotenv()

In [2]:
model = langchain.chat_models.ChatOpenAI(model_name='gpt-3.5-turbo')

In [3]:
import langchain
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()

In [30]:
@dataclass
class TeaAction:
    kind_of_tea: str = Field(
        description="Name of tea that can be made. It should be 'fruit' or 'green'")    

a1 = Affordance("go_to_outside", "You leave the room")
a2 = Affordance("make_tea", "You make tea; You may choose between fruit or green tea.", TeaAction)
with TracingNode("root") as root:
    query_for_action(model, "You are thirsty, what you want to do?", [a1, a2])
root.display()

In [ ]:
@dataclass
class TeaAction:
    kind_of_tea: str = Field(
        description="Name of tea that can be made. It should be 'fruit' or 'green'")  


class Hall(DelayedEnvironment):
    def __init__(self, parent, affs):
        aff = Affordance("look_at_window", "You can look outside", environment=self)
        super().__init__([], parent)
        self.affordances = affs + (aff,)

    def _step(self):
         for actor in self.actors:
             self.perform_action(actor.act(self.affordances))
         for action in self.take_delayed_actions():
             assert action.affordance.name == "look_at_window"
             action.origin.obseve("You may look from the window in the hall and you see that it is rainy day outside")
             self.perform_action(action)


class Kitchen(DelayedEnvironment):

    def __init__(self, parent, affs):
        super().__init__([], parent)
        aff = Affordance("make_tea", "You may make tea; You may choose between fruit or green tea.", TeaAction, environment=self)        
        super().__init__([], parent)
        self.affordances = affs + (aff,)

    def _step(self):
         for actor in self.actors:
             self.perform_action(actor.act(self.affordances))
         for action in self.take_delayed_actions():
             assert action.affordance.name == "make_tea"
             action.origin.observe(f"You made {action.args.kind_of_tea}")             


class World(DelayedEnvironment):
    def __init__(self):        
        super().__init__([])
        self.hall = Hall(self, (Affordance("go_to_kitchen", "You may go to kitchen", environment=self),))
        self.kitchen = Kitchen(self, (Affordance("go_to_hall", "You may go to hall", environment=self),))
        self.envs = [self.hall, self.kitchen]

    def add_actor(self, actor):
        self.hall.add_actor(actor)
        super().add_actor(actor)

    def _step(self):
        for env in self.envs:
            env.step()
        for action in self.take_delayed_actions():
            assert action.affordance.name in ("go_to_kitchen", "go_to_hall")            
            if action.affordance.name == "go_to_kitchen":
                action.origin.observe("You went to kitchen")
                self.hall.remove_actor(action.origin)
                self.kitchen.add_actor(action.origin)
            else:
                action.origin.observe("You went to hall")
                self.kitchen.remove_actor(action.origin)
                self.hall.actors.add_actor(action.origin)                

actor = OneShotLLMActor("TeaMaker", model=model, initial_prompt="You want to make a tea")

with TracingNode("root") as root:
    world = World()    
    world.add_actor(actor)
    world.step()
    world.step()

root.display()